<a href="https://colab.research.google.com/github/Renevelasquezcarcamo/risk-tool-project/blob/main/main_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pysentimiento
!pip install pandas
!pip install unidecode
!pip install nltk

zsh:1: command not found: pip
zsh:1: command not found: pip
zsh:1: command not found: pip
zsh:1: command not found: pip


In [ ]:
import re
from re import search
from nltk.corpus import stopwords
import string
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet
sentiment_analyzer = create_analyzer(task="sentiment", lang="es")
# emotion_analyzer = create_analyzer(task="emotion", lang="es")
# hate_analyzer = create_analyzer(task="hate_speech", lang="es")
from unidecode import unidecode

In [ ]:
df = pd.read_excel('/content/export.xlsx')
df['date'] = pd.to_datetime(df['fecha'])
df['hour'] = df['date'].dt.hour
df['date2'] = df['date'].dt.date
df = df[['texto', 'date2', 'hour']]
df.columns = ['tweet', 'date', 'hour']
name = 'Mall_Plaza'
df_tweets = df
df_tweets

In [ ]:
# Importamos nltk para extraer stopwords 
import nltk 
nltk.download('stopwords')
# Cargar stopwords en español
stopwords_es = nltk.corpus.stopwords.words('spanish')

In [ ]:
# Cargar stopwords en español
stopwords_es = nltk.corpus.stopwords.words('spanish')

"""
Excluir menciones, emails, URLs y simbolos
"""
def clean_tweet(tweet):
    # Convertir a minusculas
    tweet = tweet.lower()

    # Excluir menciones o emails
    tweet = re.sub(r'\w*@(\w+\.*\w+\.*\w+)',' ', tweet)

    # Excluir simbolos
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))

    # Excluir URLs
    tweet = re.sub(r'(?:www\.|https?)[^\s]+', ' ', tweet, flags=re.MULTILINE)

    # Borrar espacios
    tweet = tweet.strip()

    # Considerar solo valores alfa numericos
    tweet_alfa = re.compile("^(?![0-9]*$)[a-zA-Z0-9]+$")

    # Eliminar stopwords y palabras con longitud <= 2
    tokens = tweet.split()
    text = [token for token in tokens if token not in stopwords_es and len(token)>2 and tweet_alfa.match(token)]
    return ' '.join(text)

# Aplicar filtro a tweets
df_tweets['tweet_cleaned'] = df_tweets['tweet'].apply(lambda x: clean_tweet(str(x)))
df_tweets['tweet_cleaned'] = df_tweets['tweet_cleaned'].apply(lambda x: preprocess_tweet(x))

In [ ]:
#Identificación de subtemas dentro de los temas
df_topic = pd.read_excel("/content/risk.xlsx", sheet_name="versión 3")
df_filtro = df_topic.loc[0,:].reset_index()
df_filtro.columns = ['tema', 'subtema']
tema = df_filtro.tema.to_list()
for i, t in enumerate(tema):
    if 'Unnamed:' in t:
        tema[i] = tema[i-1]
df_filtro.tema = tema
df_filtro = df_filtro.dropna()

In [ ]:
#Identificación de palabras clave por subtema
df_topic = pd.read_excel("/content/risk.xlsx", sheet_name="versión 3", header=[1])
for i in range(len(df_topic.columns)):
    subtema = (df_topic.columns[i])
    words = df_topic[df_topic.columns[i]].dropna().to_list()
    words = [w.lower() for w in words]
    pattern = '|'.join(words)
    topic = df_filtro[df_filtro['subtema'] == str(subtema)].tema.to_list()[0]
    column_name = 'isin_' + str(df_topic.columns[i]) + '_' + str(topic)
    df_tweets[column_name] = df_tweets['tweet_cleaned'].str.contains(pattern)

In [ ]:
#Identificar los temas mencionados en los tweets de acuerdo a palabras clave
columns = df_tweets.columns
columns_subtopic = [lis for lis in columns if 'isin' in lis]
is_relevant = []
for i in range(len(df_tweets)):
    is_relevant.append(any(df_tweets[columns_subtopic].loc[i,:].to_list()))
df_tweets['relevant'] = is_relevant

In [ ]:
#Identificación de sentimiento
df_tweets['sentiment'] = df_tweets['tweet_cleaned'].apply(lambda x: sentiment_analyzer.predict(x).output)
# df_tweets['hate'] = df_tweets['tweet_cleaned'].apply(lambda x: hate_analyzer.predict(x).output)

In [ ]:
item = 'isin_Covid_Salud'
sentiment_options = ['POS', 'NEG', 'NEU']
df_aux = df_tweets[df_tweets[item] == True][['date', 'hour', 'sentiment']]
df_aux['date'] = pd.to_datetime(df_aux['date'], format='%Y-%m-%d')
df_aux['hour'] = pd.to_datetime(df_aux['hour'], format='%H')
df_aux['time'] = pd.to_datetime(df_aux.date.astype(str) + ' ' + df_aux.hour.astype(str)).dt.strftime("%Y-%m-%d %H:%M:%S")
pd.get_dummies(df_aux, columns = ['sentiment']).groupby('time').sum()

In [ ]:
df_tweets

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                       model="Recognai/bert-base-spanish-wwm-cased-xnli",
                       candidate_labels=["energía", "robos", "asaltos", "atropello", "accidente de transito", "inundación",
                            "sismos", "temperatura", "incendios", "contaminación", "infraestructura", "intoxicación",
                             "higiene", "ruidos molestos", "manifestación", "transporte público"],
                        hypothesis_template="Este ejemplo es {}.")

In [ ]:
def clasificador(x):
    #x = "El autor se perfila, a los 50 años de su muerte, como uno de los grandes de su siglo"
    result = classifier(x, hypothesis_template="Este ejemplo es {}.")
    a = (result['labels'][0])
    b = (result['scores'][0]*100)
    return [a, b]

In [ ]:
df_tweets = df_tweets[df_tweets['tweet_cleaned'] != ""]
df_tweets['col1'] = df_tweets['tweet_cleaned'].apply(lambda x: clasificador(str(x)))
df_tweets[['dim','dim_prob']] = pd.DataFrame(df_tweets.col1.tolist(), index= df_tweets.index)

In [ ]:
df_tweets

In [ ]:
df_tweets.to_excel('result_tweets_MPN.xlsx' , encoding='utf8', index=False)

In [9]:
#Identificar los temas mencionados en los tweets de acuerdo a palabras clave
columns = df_tweets.columns
columns_subtopic = [lis for lis in columns if 'isin' in lis]
is_relevant = []
for i in range(len(df_tweets)):
    is_relevant.append(any(df_tweets[columns_subtopic].loc[i,:].to_list()))
df_tweets['relevant'] = is_relevant

In [10]:
#Identificación de sentimiento
df_tweets['sentiment'] = df_tweets['tweet_cleaned'].apply(lambda x: sentiment_analyzer.predict(x).output)
# df_tweets['hate'] = df_tweets['tweet_cleaned'].apply(lambda x: hate_analyzer.predict(x).output)

In [11]:
item = 'isin_Covid_Salud'
sentiment_options = ['POS', 'NEG', 'NEU']
df_aux = df_tweets[df_tweets[item] == True][['date', 'hour', 'sentiment']]
df_aux['date'] = pd.to_datetime(df_aux['date'], format='%Y-%m-%d')
df_aux['hour'] = pd.to_datetime(df_aux['hour'], format='%H')
df_aux['time'] = pd.to_datetime(df_aux.date.astype(str) + ' ' + df_aux.hour.astype(str)).dt.strftime("%Y-%m-%d %H:%M:%S")
pd.get_dummies(df_aux, columns = ['sentiment']).groupby('time').sum()

,sentiment_NEG,sentiment_NEU,sentiment_POS
time,,,
2020-12-19 01:00:00,0,2,0
2020-12-19 04:00:00,0,1,0
2020-12-19 10:00:00,0,1,0
2020-12-19 14:00:00,0,1,0
2020-12-19 23:00:00,0,1,1
2020-12-20 00:00:00,0,1,0
2020-12-20 14:00:00,0,1,0
2020-12-20 15:00:00,10,0,0
2020-12-20 16:00:00,2,0,0


In [12]:
df_tweets

,tweet,date,hour,tweet_cleaned,isin_Gobierno central_Entidades,isin_Gobierno local_Entidades,isin_Policías_Entidades,isin_Ciudadanía_Entidades,isin_Energía_Seguridad pública,isin_Robos o asaltos_Seguridad pública,...,isin_Temperaturas extremas y sequías_Medio Ambiente,isin_Incendios_Medio Ambiente,isin_Exposición Química_Medio Ambiente,isin_Salud_Salud,isin_Covid_Salud,isin_Infraestructura_Salud,isin_Alimentación_Salud,isin_Odontología_Salud,relevant,sentiment
0,RT @CruisingEste: ¡Me encanta este tipo de mor...,2020-12-25,23,encanta tipo morbo quise mall iba haber mucha ...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,NEG
1,el niño del hoodie azúl q hizo contacto visual...,2020-12-25,23,hoodie hizo contacto visual conmigo mall plaza...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,NEU
2,Alguien me dice como llego de mall del sur a p...,2020-12-25,23,alguien dice llego mall sur plaza norte,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,NEU
3,RT @tsog76: 🧄🧄🧄 АР ШАНДЫН ААГТАЙ МОНГОЛ САРМИС...,2020-12-25,23,good price good price1 shangrila mall,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,NEU
4,RT @tsog76: 🧄🧄🧄 АР ШАНДЫН ААГТАЙ МОНГОЛ САРМИС...,2020-12-25,23,good price good price1 shangrila mall,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,NEU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8368,07.40 #InfoSonora\nLOKASI PELAYANAN #SIMKELILI...,2020-12-19,0,infosonora lokasi pelayanan simkeliling dki ja...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,NEU
8369,RT @nonstandardmcd: a&amp;s plaza mcdonald's (...,2020-12-19,0,aamps plaza mcdonalds early 90s manhattan mall...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,NEU
8370,Ternyata mall tunjungan plaza surabaya terbesa...,2020-12-19,0,ternyata mall tunjungan plaza surabaya terbesa...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,NEU
8371,@jjkmivaron mall plaza,2020-12-19,0,mall plaza,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,NEU


In [13]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                       model="Recognai/bert-base-spanish-wwm-cased-xnli",
                       candidate_labels=["energía", "robos", "asaltos", "atropello", "accidente de transito", "inundación",
                            "sismos", "temperatura", "incendios", "contaminación", "infraestructura", "intoxicación",
                             "higiene", "ruidos molestos", "manifestación", "transporte público"],
                        hypothesis_template="Este ejemplo es {}.")

https://huggingface.co/Recognai/bert-base-spanish-wwm-cased-xnli/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp1sadm2wx


Downloading:   0%|          | 0.00/834 [00:00<?, ?B/s]

storing https://huggingface.co/Recognai/bert-base-spanish-wwm-cased-xnli/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/3031bfc3feed8613fb368974a959fe63c6720d92b5fa4ffea22a5ed9578413fb.99f1e77ae9c7a79c0550f7263195fc63bd63b3391f55395bd6897c6a6e7aad0a
creating metadata file for /root/.cache/huggingface/transformers/3031bfc3feed8613fb368974a959fe63c6720d92b5fa4ffea22a5ed9578413fb.99f1e77ae9c7a79c0550f7263195fc63bd63b3391f55395bd6897c6a6e7aad0a
loading configuration file https://huggingface.co/Recognai/bert-base-spanish-wwm-cased-xnli/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3031bfc3feed8613fb368974a959fe63c6720d92b5fa4ffea22a5ed9578413fb.99f1e77ae9c7a79c0550f7263195fc63bd63b3391f55395bd6897c6a6e7aad0a
Model config BertConfig {
  "_name_or_path": "Recognai/bert-base-spanish-wwm-cased-xnli",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "grad

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

storing https://huggingface.co/Recognai/bert-base-spanish-wwm-cased-xnli/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/d01043de9bb67965b36859613fdb4727f8ac21927e9cc2684cb313f9fe81da27.54a250e1e8fe69ea00b55ccecb633fe8ded0e214917b1da4886a1f96505b6941
creating metadata file for /root/.cache/huggingface/transformers/d01043de9bb67965b36859613fdb4727f8ac21927e9cc2684cb313f9fe81da27.54a250e1e8fe69ea00b55ccecb633fe8ded0e214917b1da4886a1f96505b6941
loading weights file https://huggingface.co/Recognai/bert-base-spanish-wwm-cased-xnli/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/d01043de9bb67965b36859613fdb4727f8ac21927e9cc2684cb313f9fe81da27.54a250e1e8fe69ea00b55ccecb633fe8ded0e214917b1da4886a1f96505b6941
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at Recognai/bert-base-spanish-wwm-cased-x

...

storing https://huggingface.co/Recognai/bert-base-spanish-wwm-cased-xnli/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/e0f4e1a779da22022cf4083ee9c9915df6857b9662b412ee0bd1d6265aee5c0b.c2d4f53f5d642ae24a7063e992e91b21d486369029581099ed72bdafee998005
creating metadata file for /root/.cache/huggingface/transformers/e0f4e1a779da22022cf4083ee9c9915df6857b9662b412ee0bd1d6265aee5c0b.c2d4f53f5d642ae24a7063e992e91b21d486369029581099ed72bdafee998005
loading configuration file https://huggingface.co/Recognai/bert-base-spanish-wwm-cased-xnli/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3031bfc3feed8613fb368974a959fe63c6720d92b5fa4ffea22a5ed9578413fb.99f1e77ae9c7a79c0550f7263195fc63bd63b3391f55395bd6897c6a6e7aad0a
Model config BertConfig {
  "_name_or_path": "Recognai/bert-base-spanish-wwm-cased-xnli",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": nul

Downloading:   0%|          | 0.00/236k [00:00<?, ?B/s]

storing https://huggingface.co/Recognai/bert-base-spanish-wwm-cased-xnli/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/85f914bdec01b32436ee9849212b8387cef0a0b48d028aa1e6f8256b74d7b7e8.13d9106372e0c2be5cfd538f29343b96f32ad81a28e926b2646905bab05f5f3e
creating metadata file for /root/.cache/huggingface/transformers/85f914bdec01b32436ee9849212b8387cef0a0b48d028aa1e6f8256b74d7b7e8.13d9106372e0c2be5cfd538f29343b96f32ad81a28e926b2646905bab05f5f3e
https://huggingface.co/Recognai/bert-base-spanish-wwm-cased-xnli/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpomercsz5


Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

storing https://huggingface.co/Recognai/bert-base-spanish-wwm-cased-xnli/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/3bceeaee6bfc243e7d04e83dce69380dfa4ac5d347eae32ae14b70c6a626b479.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /root/.cache/huggingface/transformers/3bceeaee6bfc243e7d04e83dce69380dfa4ac5d347eae32ae14b70c6a626b479.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/Recognai/bert-base-spanish-wwm-cased-xnli/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/85f914bdec01b32436ee9849212b8387cef0a0b48d028aa1e6f8256b74d7b7e8.13d9106372e0c2be5cfd538f29343b96f32ad81a28e926b2646905bab05f5f3e
loading file https://huggingface.co/Recognai/bert-base-spanish-wwm-cased-xnli/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/Recognai/bert-base-spanish-wwm-cased-xnli/resolve/main/added_tokens.jso

In [14]:
def clasificador(x):
    #x = "El autor se perfila, a los 50 años de su muerte, como uno de los grandes de su siglo"
    result = classifier(x, hypothesis_template="Este ejemplo es {}.")
    a = (result['labels'][0])
    b = (result['scores'][0]*100)
    return [a, b]

In [15]:
df_tweets = df_tweets[df_tweets['tweet_cleaned'] != ""]
df_tweets['col1'] = df_tweets['tweet_cleaned'].apply(lambda x: clasificador(str(x)))
df_tweets[['dim','dim_prob']] = pd.DataFrame(df_tweets.col1.tolist(), index= df_tweets.index)

In [18]:
df_tweets

,tweet,date,hour,tweet_cleaned,isin_Gobierno central_Entidades,isin_Gobierno local_Entidades,isin_Policías_Entidades,isin_Ciudadanía_Entidades,isin_Energía_Seguridad pública,isin_Robos o asaltos_Seguridad pública,...,isin_Covid_Salud,isin_Infraestructura_Salud,isin_Alimentación_Salud,isin_Odontología_Salud,relevant,sentiment,col1,dim,dim_prob,dim2
0,RT @CruisingEste: ¡Me encanta este tipo de mor...,2020-12-25,23,encanta tipo morbo quise mall iba haber mucha ...,False,False,False,False,False,False,...,False,False,False,False,False,NEG,"[asaltos, 19.48051154613495]",asaltos,19.480512,a
1,el niño del hoodie azúl q hizo contacto visual...,2020-12-25,23,hoodie hizo contacto visual conmigo mall plaza...,False,False,False,False,False,False,...,False,False,False,False,False,NEU,"[manifestación, 32.29040801525116]",manifestación,32.290408,m
2,Alguien me dice como llego de mall del sur a p...,2020-12-25,23,alguien dice llego mall sur plaza norte,False,False,False,False,False,False,...,False,False,False,False,False,NEU,"[manifestación, 30.439648032188416]",manifestación,30.439648,m
3,RT @tsog76: 🧄🧄🧄 АР ШАНДЫН ААГТАЙ МОНГОЛ САРМИС...,2020-12-25,23,good price good price1 shangrila mall,False,False,False,False,False,False,...,False,False,False,False,False,NEU,"[manifestación, 41.69960916042328]",manifestación,41.699609,m
4,RT @tsog76: 🧄🧄🧄 АР ШАНДЫН ААГТАЙ МОНГОЛ САРМИС...,2020-12-25,23,good price good price1 shangrila mall,False,False,False,False,False,False,...,False,False,False,False,False,NEU,"[manifestación, 41.69960916042328]",manifestación,41.699609,m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8368,07.40 #InfoSonora\nLOKASI PELAYANAN #SIMKELILI...,2020-12-19,0,infosonora lokasi pelayanan simkeliling dki ja...,False,False,False,False,False,False,...,False,False,False,False,False,NEU,"[asaltos, 30.65105974674225]",asaltos,30.651060,a
8369,RT @nonstandardmcd: a&amp;s plaza mcdonald's (...,2020-12-19,0,aamps plaza mcdonalds early 90s manhattan mall...,False,False,False,False,False,False,...,False,False,False,False,False,NEU,"[manifestación, 24.205169081687927]",manifestación,24.205169,m
8370,Ternyata mall tunjungan plaza surabaya terbesa...,2020-12-19,0,ternyata mall tunjungan plaza surabaya terbesa...,False,False,False,False,False,False,...,False,False,False,False,False,NEU,"[manifestación, 30.385905504226685]",manifestación,30.385906,m
8371,@jjkmivaron mall plaza,2020-12-19,0,mall plaza,False,False,False,False,False,False,...,False,False,False,False,False,NEU,"[infraestructura, 27.72284746170044]",infraestructura,27.722847,i


In [19]:
  df_tweets.to_excel('result_tweets_MPN.xlsx' , encoding='utf8', index=False)